In [1]:
import netCDF4
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import datetime


In [2]:
nc_file = "OMI-Aura_L3-OMNO2d_2004m1001_v003-2019m1121t082956.he5.SUB.nc4"

In [3]:
nc = netCDF4.Dataset("./data/"+nc_file, mode='r')


In [4]:
nc.variables.keys()

dict_keys(['XDim', 'YDim', 'ColumnAmountNO2Trop'])

In [5]:
nc.variables

{'XDim': <class 'netCDF4._netCDF4.Variable'>
 float64 XDim(XDim)
     standard_name: longitude
     long_name: longitude
     units: degrees_east
     axis: X
 unlimited dimensions: 
 current shape = (295,)
 filling off, 'YDim': <class 'netCDF4._netCDF4.Variable'>
 float64 YDim(YDim)
     standard_name: latitude
     long_name: latitude
     units: degrees_north
     axis: Y
 unlimited dimensions: 
 current shape = (105,)
 filling off, 'ColumnAmountNO2Trop': <class 'netCDF4._netCDF4.Variable'>
 float32 ColumnAmountNO2Trop(YDim, XDim)
     _FillValue: -1.2676506e+30
     missing_value: -1.2676506e+30
     Description: Field=ColumnAmountNO2Trop,StdField=ColumnAmountNO2TropStd,SolarZenithAngle=[0:85],VcdQualityFlags=~19,XTrackQualityFlags=~[1:254],TerrainReflectivity=[0:1000]
     MissingValue: -1.2676506e+30
     Offset: 0.0
     ScaleFactor: 1.0
     Title: NO2 tropospheric column density
     Units: molec/cm2
     fullnamepath: /HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2T

In [6]:
lon = nc.variables["XDim"][:]
lat = nc.variables["YDim"][:]
NO2 = nc.variables["ColumnAmountNO2Trop"][:]

In [7]:
date_parse = nc_file[31-7:33-7]  + "/" + nc_file[33-7:35-7] +  "/" + nc_file[26-7:30-7]
format_str = '%m/%d/%Y' # The format
datetime_obj = datetime.datetime.strptime(date_parse, format_str)
date = datetime_obj.date()

In [8]:
lon.shape

(295,)

In [9]:
lat.shape

(105,)

In [10]:
NO2.shape

(105, 295)

In [11]:
import os
import xarray as xr
nc = xr.open_dataset("./data/"+nc_file)
df = nc.ColumnAmountNO2Trop.to_dataframe()
#.to_csv(nc_file+'.csv')


In [12]:
df.head()

ColumnAmountNO2Trop
YDim   XDim                         
24.625 -132.375         7.397606e+14
       -132.125         5.015970e+14
       -131.875         6.355413e+14
       -131.625         8.383936e+14
       -131.375         6.277493e+14

In [13]:
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [14]:
df = df.reset_index()

In [15]:
df['Date'] = date

In [16]:
df.index.name = 'id'

In [19]:
df = df.rename(columns={'YDim': 'lat', 'XDim': 'lon'})

In [20]:
df.head()

,lat,lon,ColumnAmountNO2Trop,Date
id,,,,
0,24.625,-132.375,7.397606e+14,2004-10-01
1,24.625,-132.125,5.015970e+14,2004-10-01
2,24.625,-131.875,6.355413e+14,2004-10-01
3,24.625,-131.625,8.383936e+14,2004-10-01
4,24.625,-131.375,6.277493e+14,2004-10-01


In [ ]:
"""features = df.apply(
    lambda row: Feature(geometry=Point((float(row['YDim']), float(row['XDim'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = df.drop(['YDim', 'XDim'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)
"""

In [ ]:
"""with open("Finished/" + nc_file + ".json", 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)
    """

In [21]:
df.to_csv("Finished/" + nc_file + ".csv")